In [ ]:
# устанавливаем необходимые зависимости

!pip3 install transformers==4.22.1
!pip3 install torch==1.7.0+cu92
!pip3 install opencv-python==4.5.4
!pip3 install segments-ai
!pip3 install imutils==0.5.4
!pip3 install pathlib
!pip3 install Pillow==7.0.0
!pip3 install numpy==1.21.6
!pip3 install pandas==0.25.3
!pip3 install requests
!pip3 install matplotlib==3.1.2

In [1]:
import os
import requests


from pathlib import Path
import numpy as np

# без этого падает обучение
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
from torch import nn
from torchvision.transforms import ColorJitter

from segments import SegmentsClient
from segments.typing import Release
from segments.huggingface import release2dataset
from segments.utils import get_semantic_bitmap

import transformers
from datasets import load_metric
from transformers import TrainingArguments
from transformers import SegformerForSemanticSegmentation
from transformers import Trainer
from transformers import SegformerFeatureExtractor

import matplotlib.pyplot as plt
from PIL import Image,ImageDraw

In [2]:
# подключаемся к segments для скачивания размеченного dataset
client = SegmentsClient("d006ca0471e21f635ae19a9a8e2e5788e0a0d2f4")
release = client.get_release("timekeeper/license_plates", "v0.6")

# конвертируем в huggingface Dataset
hf_dataset = release2dataset(release)

  0%|          | 0/999 [00:00<?, ?ex/s]

  0%|          | 0/999 [00:00<?, ?ex/s]

In [3]:
hf_dataset

Dataset({
    features: ['name', 'uuid', 'status', 'image', 'label.annotations', 'label.segmentation_bitmap'],
    num_rows: 999
})

In [4]:
bkp_dataset = hf_dataset

In [5]:
hf_dataset = bkp_dataset

In [6]:
# выбираем из датасета только размеченные изображения
filtered = []
for i in range(len(hf_dataset)):
    if hf_dataset[i].get('status') in ['UNLABELED']:
        filtered.append(i)
hf_dataset = hf_dataset.select(
    (
        i for i in range(len(hf_dataset)) if i not in set(filtered)
    )
)

Parameter 'indices'=<generator object <genexpr> at 0x7f52e37aac78> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [7]:
hf_dataset

Dataset({
    features: ['name', 'uuid', 'status', 'label.annotations', 'image', 'label.segmentation_bitmap'],
    num_rows: 291
})

In [8]:
# преобразуем в битмап
def convert_segmentation_bitmap(example):
    return {
        "label.segmentation_bitmap":
            get_semantic_bitmap(
                example["label.segmentation_bitmap"],
                example["label.annotations"],
                id_increment=0,
            )
    }


semantic_dataset = hf_dataset.map(
    convert_segmentation_bitmap,
)

  0%|          | 0/291 [00:00<?, ?ex/s]

In [9]:
# переименовываем колонки в соотетствии с форматом huggingface
semantic_dataset = semantic_dataset.rename_column('image', 'pixel_values')
semantic_dataset = semantic_dataset.rename_column('label.segmentation_bitmap', 'label')
semantic_dataset = semantic_dataset.remove_columns(['name', 'uuid', 'status', 'label.annotations'])

In [10]:
semantic_dataset[0]

{'pixel_values': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=400x300 at 0x7F52E23332E8>,
 'label': <PIL.PngImagePlugin.PngImageFile image mode=L size=400x300 at 0x7F52E23337B8>}

In [11]:
# разбиваем на train/test
ds = semantic_dataset.train_test_split(test_size=0.2)
dirt_train_ds = ds["train"]
ds_tr = ds["test"].train_test_split(test_size=0.4)
dirt_test_ds = ds_tr["train"]
dirt_val_ds = ds_tr["test"]

In [12]:
dirt_train_ds

Dataset({
    features: ['pixel_values', 'label'],
    num_rows: 232
})

In [13]:
feature_extractor = SegformerFeatureExtractor()
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 

def train_transforms(example_batch):
    images = [jitter(x) for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [x for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs


# Set transforms
dirt_train_ds.set_transform(train_transforms)
dirt_test_ds.set_transform(val_transforms)
dirt_val_ds.set_transform(val_transforms)

In [14]:
# чистим от ошибок при feature_extraction
train_ds = []
indexes = []
for i in range(len(dirt_train_ds)):
    try:
        dirt_train_ds[i]
    except:
        indexes.append(i)

train_ds = dirt_train_ds.select(
    (
        i for i in range(len(dirt_train_ds)) 
        if i not in set(indexes)
    )
)

In [19]:
train_ds

Dataset({
    features: ['pixel_values', 'label'],
    num_rows: 205
})

In [15]:
test_ds = []
indexes = []
for i in range(len(dirt_test_ds)):
    try:
        dirt_test_ds[i]
    except:
        indexes.append(i)
test_ds = dirt_test_ds.select(
    (
        i for i in range(len(dirt_test_ds)) 
        if i not in set(indexes)
    )
)

In [18]:
test_ds

Dataset({
    features: ['pixel_values', 'label'],
    num_rows: 29
})

In [16]:
val_ds = []
indexes = []
for i in range(len(dirt_val_ds)):
    try:
        dirt_val_ds[i]
    except:
        indexes.append(i)
val_ds = dirt_val_ds.select(
    (
        i for i in range(len(dirt_val_ds)) 
        if i not in set(indexes)
    )
)

In [17]:
val_ds

Dataset({
    features: ['pixel_values', 'label'],
    num_rows: 23
})

### Fine tune

In [20]:
# базовая модель
pretrained_model_name = "nvidia/mit-b0" 
num_labels = 5

id2label={0: 'unlabeled',
          1: 'license plate',
          2: 'car',
          3: 'truck',
          4: 'emergency'}
label2id={'unlabeled':0,
          'license plate':1,
          'car': 2,
          'truck': 3,
          'emergency': 4}



model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.li

In [21]:
epochs = 300
lr = 0.00004
batch_size = 3 #4
# {'learning_rate': 4e-05, 'batch_size': 3, 'weight_decay': 0.0, 'adafactor': False}

training_args = TrainingArguments(
    "segformer-b0-finetuned-segments-license-plate",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=100, #3
    eval_steps=100, #3
    logging_steps=1,
    eval_accumulation_steps=10, #3
    load_best_model_at_end=True,
    push_to_hub=False,
    hub_strategy="end",
#     report_to="wandb"
)

# os.environ["WANDB_API_KEY"] = 'a427b2ba60f438b85345fe30f56d9f8fd6802294'
# wandb.login(anonymous='must')
# wandb.init()

In [22]:
# подготовливаем расчет метрик
metric = load_metric("mean_iou")

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)
    # преобразовываем логиты по размеру картинки
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    metrics = metric.compute(predictions=pred_labels, references=labels, 
                                   num_labels=num_labels, 
                                   ignore_index=0,
                                   reduce_labels=feature_extractor.reduce_labels)
    for key, value in metrics.items():
      if type(value) is np.ndarray:
        metrics[key] = value.tolist()
    return metrics

def collate_fn(examples):
#     print(examples)
    pixel_values = torch.tensor([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["labels"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

/home/ewp/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


In [23]:
# инициализируем тейнер
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

In [24]:
# Проверка на доступность CUDA

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

torch.backends.cudnn.benchmark=True
torch.backends.cudnn.deterministic=False

True
0
1
GeForce RTX 2060


In [25]:
# обучение
trainer.train()

/home/ewp/.local/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 205
  Num Epochs = 1
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 3
  Gradient Accumulation steps = 1
  Total optimization steps = 69
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anony-mouse-411946. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=69, training_loss=1.1899675752805627, metrics={'train_runtime': 40.7702, 'train_samples_per_second': 5.028, 'train_steps_per_second': 1.692, 'total_flos': 3593976678973440.0, 'train_loss': 1.1899675752805627, 'epoch': 1.0})

In [27]:
# тестирование
trainer.predict(test_ds)[-1]

***** Running Prediction *****
  Num examples = 29
  Batch size = 3


/home/ewp/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/d4add40cf977cdd73590b5873fa830f3f13adb678f6777a29fb07b7c81d14342/mean_iou.py:259: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label


{'test_loss': 0.9694570302963257,
 'test_mean_iou': 0.31029391236881493,
 'test_mean_accuracy': 0.45156642285353144,
 'test_overall_accuracy': 0.7560240617387859,
 'test_per_category_iou': [0.0,
  0.1499825406802151,
  0.8545607977281529,
  0.34264882943143815,
  0.20427739400426842],
 'test_per_category_accuracy': [nan,
  0.15264983509609917,
  0.9246327290655393,
  0.5002278527160043,
  0.22875527453648295],
 'test_runtime': 27.6347,
 'test_samples_per_second': 1.049,
 'test_steps_per_second': 0.362}

### Подбор параметров

In [28]:
def get_tainer(lgrid, model, train_ds, val_ds, compute_metrics):
    training_args = TrainingArguments(
                "segformer-b0-finetuned-segments-license-plate",
                learning_rate=lgrid['learning_rate'],
                num_train_epochs=200,
                per_device_train_batch_size=lgrid['batch_size'],
                per_device_eval_batch_size=lgrid['batch_size'],
                save_total_limit=3,
                evaluation_strategy="steps",
                save_strategy="steps",
                save_steps=100, #3
                eval_steps=100, #3
                logging_steps=1,
                eval_accumulation_steps=10, #3
                load_best_model_at_end=True,
                push_to_hub=False,
                hub_strategy="end",
                adafactor=lgrid['adafactor']
            )
    
    
    return Trainer(
                model=model,
                args=training_args,
                train_dataset=train_ds,
                eval_dataset=val_ds,
                compute_metrics=compute_metrics,
            )

In [29]:
# создаем сетку параметров
grid = {'learning_rate': [0.00004, 0.00001, 0.0001],
        'batch_size': [3],
        'weight_decay':[0.0, 0.1, 0.01],
        'adafactor': [True, False]}

In [30]:
from itertools import product

def process(d):
    to_product = []  # [[('a', 1), ('a', 2)], [('b', 3),], ...]
    for k, v in d.items():
        if isinstance(v, list):
            to_product.append([(k, i) for i in v])
        elif isinstance(v, dict):
            to_product.append([(k, i) for i in process(v)])
        else:
            to_product.append([(k, v)])
    return [dict(l) for l in product(*to_product)]

In [ ]:
# пробегаемся по сетке, фиксируем результаты обучения в массивчик
results = []
for i in process(grid)[:]:
    print(i)
    
    # базовая модель
    pretrained_model_name = "nvidia/mit-b0" 
    num_labels = 5

    id2label={0: 'unlabeled',
              1: 'license plate',
              2: 'car',
              3: 'truck',
              4: 'emergency'}
    label2id={'unlabeled':0,
              'license plate':1,
              'car': 2,
              'truck': 3,
              'emergency': 4}



    model = SegformerForSemanticSegmentation.from_pretrained(
        pretrained_model_name,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id
    )
    
    trainer = get_tainer(i, model, train_ds, val_ds, compute_metrics)
    trainer.train()
    metrics = trainer.predict(test_ds)[-1]
    results.append([{**i, **metrics}])

{'learning_rate': 4e-05, 'batch_size': 3, 'weight_decay': 0.0, 'adafactor': True}


loading configuration file config.json from cache at /home/ewp/.cache/huggingface/hub/models--nvidia--mit-b0/snapshots/ed0b85c75627eab6a3c6989627450cf95f115381/config.json
Model config SegformerConfig {
  "architectures": [
    "SegformerForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout_prob": 0.1,
  "decoder_hidden_size": 256,
  "depths": [
    2,
    2,
    2,
    2
  ],
  "downsampling_rates": [
    1,
    4,
    8,
    16
  ],
  "drop_path_rate": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_sizes": [
    32,
    64,
    160,
    256
  ],
  "id2label": {
    "0": "unlabeled",
    "1": "license plate",
    "2": "car",
    "3": "truck",
    "4": "emergency"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "label2id": {
    "car": 2,
    "emergency": 4,
    "license plate": 1,
    "truck": 3,
    "unlabeled": 0
  },
  "layer_norm_eps": 1e-06,
  "mlp_ratios": [
    4,
    4,
    4,
    4
  ],
  "model_type": "segfor

Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Per Category Iou,Per Category Accuracy
100,0.858600,0.499254,0.483487,0.689494,0.886732,"[0.0, 0.6055552208230404, 0.932104253427542, 0.5543971691058858, 0.3253780513481045]","[nan, 0.6958086983010482, 0.9493240904589283, 0.6969784449576597, 0.4158628986302526]"
200,0.600900,0.405735,0.497066,0.703240,0.882049,"[0.0, 0.6474526414872338, 0.9255269651525405, 0.6094355897529846, 0.30291343452198344]","[nan, 0.7619252018111076, 0.9348430229180661, 0.7879680622174848, 0.32822228319368524]"
300,0.378700,0.328913,0.521786,0.708646,0.905649,"[0.0, 0.655089585529057, 0.9537675977088004, 0.6059274036754636, 0.394146424172306]","[nan, 0.7401448332202545, 0.9699633251110804, 0.7074475434752571, 0.4170289778603238]"
400,0.361300,0.292755,0.532147,0.735573,0.912201,"[0.0, 0.6813547289440507, 0.9614599824827521, 0.649361566966649, 0.3685608881803888]","[nan, 0.7707730438515114, 0.9683875189192405, 0.7566495923886224, 0.4464817120786813]"
500,0.315300,0.314654,0.429661,0.662561,0.854844,"[0.0, 0.7102703521011265, 0.9538238884057404, 0.18820530541144392, 0.29600436442989636]","[nan, 0.7795655003392366, 0.9587650132065082, 0.19620368725449558, 0.7157088284333383]"
600,0.217400,0.313929,0.437367,0.673278,0.875270,"[0.0, 0.7189047518701928, 0.9670024878736313, 0.19241715448469782, 0.3085119342852916]","[nan, 0.7558050982401241, 0.9806669799726513, 0.20835553976431576, 0.7482851776028365]"
700,0.344600,0.368710,0.428572,0.692010,0.867322,"[0.0, 0.7317740795476451, 0.9675864458754347, 0.10902714466370614, 0.33447298724638597]","[nan, 0.7884548815441492, 0.9742440913355358, 0.10976638834606502, 0.8955752305776578]"
800,0.583700,0.312628,0.423003,0.642494,0.861037,"[0.0, 0.7438687576307149, 0.9731817932764099, 0.1740504901908147, 0.22391415914861434]","[nan, 0.8689300895861315, 0.9762011521821785, 0.19996644361540436, 0.5248781157003862]"
900,0.087300,0.227853,0.573189,0.763051,0.934055,"[0.0, 0.7604276246581106, 0.982514979934371, 0.6813316960362034, 0.44167131033142987]","[nan, 0.8007227814624556, 0.98655103333578, 0.8090839107005389, 0.45584728055549695]"
1000,0.254200,0.213894,0.544993,0.744538,0.924845,"[0.0, 0.7573716805979215, 0.9776634634010415, 0.6112989948108744, 0.3786301417499446]","[nan, 0.7941595934700434, 0.9849373103945116, 0.7394568800457946, 0.4595987843228298]"


***** Running Evaluation *****
  Num examples = 23
  Batch size = 3
Trainer is attempting to log a value of "[0.0, 0.6055552208230404, 0.932104253427542, 0.5543971691058858, 0.3253780513481045]" of type <class 'list'> for key "eval/per_category_iou" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[nan, 0.6958086983010482, 0.9493240904589283, 0.6969784449576597, 0.4158628986302526]" of type <class 'list'> for key "eval/per_category_accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to segformer-b0-finetuned-segments-license-plate/checkpoint-100
Configuration saved in segformer-b0-finetuned-segments-license-plate/checkpoint-100/config.json
Model weights saved in segformer-b0-finetuned-segments-license-plate/checkpoint-100/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 23
  Batch

Saving model checkpoint to segformer-b0-finetuned-segments-license-plate/checkpoint-700
Configuration saved in segformer-b0-finetuned-segments-license-plate/checkpoint-700/config.json
Model weights saved in segformer-b0-finetuned-segments-license-plate/checkpoint-700/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-license-plate/checkpoint-500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 23
  Batch size = 3
/home/ewp/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/d4add40cf977cdd73590b5873fa830f3f13adb678f6777a29fb07b7c81d14342/mean_iou.py:259: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label
Trainer is attempting to log a value of "[0.0, 0.7438687576307149, 0.9731817932764099, 0.1740504901908147, 0.22391415914861434]" of type <class 'list'> for key "eval/per_category_iou" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we d

...

Trainer is attempting to log a value of "[nan, 0.9042799185832375, 0.9800262216933203, 0.8575235388168414, 0.5124047614022499]" of type <class 'list'> for key "eval/per_category_accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to segformer-b0-finetuned-segments-license-plate/checkpoint-8300
Configuration saved in segformer-b0-finetuned-segments-license-plate/checkpoint-8300/config.json
Model weights saved in segformer-b0-finetuned-segments-license-plate/checkpoint-8300/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-license-plate/checkpoint-8100] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 23
  Batch size = 3
/home/ewp/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/d4add40cf977cdd73590b5873fa830f3f13adb678f6777a29fb07b7c81d14342/mean_iou.py:259: RuntimeWarning: invalid value encountered in true_divide
  acc = total_are

Saving model checkpoint to segformer-b0-finetuned-segments-license-plate/checkpoint-8900
Configuration saved in segformer-b0-finetuned-segments-license-plate/checkpoint-8900/config.json
Model weights saved in segformer-b0-finetuned-segments-license-plate/checkpoint-8900/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-license-plate/checkpoint-8700] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 23
  Batch size = 3
/home/ewp/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/d4add40cf977cdd73590b5873fa830f3f13adb678f6777a29fb07b7c81d14342/mean_iou.py:259: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label
Trainer is attempting to log a value of "[0.0, 0.8664562228415199, 0.9784040865920681, 0.3352708979916675, 0.28595903815486057]" of type <class 'list'> for key "eval/per_category_iou" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so 

Deleting older checkpoint [segformer-b0-finetuned-segments-license-plate/checkpoint-9300] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 23
  Batch size = 3
/home/ewp/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/d4add40cf977cdd73590b5873fa830f3f13adb678f6777a29fb07b7c81d14342/mean_iou.py:259: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label
Trainer is attempting to log a value of "[0.0, 0.8702325643312611, 0.9766112455631689, 0.32976466459059417, 0.2860017278253164]" of type <class 'list'> for key "eval/per_category_iou" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[nan, 0.9051522410379252, 0.9782657017955494, 0.36396340380174097, 0.6794917794052469]" of type <class 'list'> for key "eval/per_category_accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is inco

Trainer is attempting to log a value of "[nan, 0.9079492114481937, 0.9794411210278292, 0.3581379167406881, 0.6767164053101454]" of type <class 'list'> for key "eval/per_category_accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to segformer-b0-finetuned-segments-license-plate/checkpoint-10200
Configuration saved in segformer-b0-finetuned-segments-license-plate/checkpoint-10200/config.json
Model weights saved in segformer-b0-finetuned-segments-license-plate/checkpoint-10200/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-license-plate/checkpoint-10000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 23
  Batch size = 3
/home/ewp/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/d4add40cf977cdd73590b5873fa830f3f13adb678f6777a29fb07b7c81d14342/mean_iou.py:259: RuntimeWarning: invalid value encountered in true_divide
  acc = total

Saving model checkpoint to segformer-b0-finetuned-segments-license-plate/checkpoint-10800
Configuration saved in segformer-b0-finetuned-segments-license-plate/checkpoint-10800/config.json
Model weights saved in segformer-b0-finetuned-segments-license-plate/checkpoint-10800/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-license-plate/checkpoint-10600] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 23
  Batch size = 3
/home/ewp/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/d4add40cf977cdd73590b5873fa830f3f13adb678f6777a29fb07b7c81d14342/mean_iou.py:259: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label
Trainer is attempting to log a value of "[0.0, 0.8637821355344093, 0.9794090548065045, 0.4177082944666244, 0.2899762919865913]" of type <class 'list'> for key "eval/per_category_iou" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect 

## inference

In [ ]:
BASE_DIR = Path().cwd()

In [ ]:
# поднимаем модель с диска
model_dir = BASE_DIR.joinpath('segformer-b0-finetuned-segments-license-plate', 'checkpoint-13800')

In [ ]:


# инициализируем модель и feture extractor
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/mit-b0")
model = SegformerForSemanticSegmentation.from_pretrained(model_dir)

In [ ]:
# возвращаем разцветку классов
def plate_palette():
    """Plate palette that maps each class to RGB values."""
    return [
        [0, 0, 0],
        [0,113,188],
        [236,176,31],
        [125,46,141],
        [118,171,47]
    ]

# наклкдываем сегментацию н аизображение и возвращаем все найденные классы
def get_seg_overlay(image, seg):
    color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8) # height, width, 3
    palette = np.array(plate_palette())
    classes = {}
    for label, color in enumerate(palette):
        
        color_seg[seg == label, :] = color
#     color_seg = color_seg[..., ::-1]
    classids = np.unique(seg).tolist()
    
    classes = {k:id2label.get(k) for k in classids}
    
    w, h = image.size[0], image.size[1]
    img1 = ImageDraw.Draw(image)
    shape = [(40, 40), (w - 10, h - 10)]
    img1.rectangle(shape, fill =None, outline ="red")
    # отрисовываем маску
    img = np.array(image) * 0.5 + color_seg * 0.5
    img = img.astype(np.uint8)


    return img, classes

In [ ]:
%%time
url = 'https://cdnstatic.rg.ru/uploads/images/gallery/9bd89d69/3_a2044431.jpg'
# url = 'https://static3.car.ru/uploaded/2017/1/2/1138/650_0b46dc17078bd6d49ecb5b7300faba02_3de63740b9735f3b4f50b9099344327d.jpg'
# url = 'https://webpulse.imgsmail.ru/imgpreview?mb=webpulse&key=pulse_cabinet-image-836541bc-c469-48b9-9be1-622650e0cf9a&fu=1&kr=1&h=344&w=630'
# url = "https://aa-highway.com.sg/wp-content/uploads/2019/02/15712966659_2285d07d64_72dpi.jpg"
# url = 'https://motor.ru/imgs/2017/03/13/21/214907/62bb46b469c38aacce8c19873623812baa4f5a1a.jpg'
# url = 'https://media.istockphoto.com/photos/russian-police-car-on-a-city-street-picture-id1147082137'
# url = 'https://www.pozhmashina.ru/wp-content/uploads/2021/10/pozharnaya-mashina-gaz-53-1.jpg'
# url = 'https://vsyatekhnika.ru/uploads/posts/2020-03/1584018689_populjarnyj-avtomobil-kamaz-65115.jpg'
# url = 'https://www.avto-master.com/upload/resize_cache/iblock/4a2/1115_665_194741b4c66f48e785505267fcb7d795f/5099aa3eb1bf01df9407cde0b21a9ee1daf457375e95c6b4db_pimgpsh_fullsize_distr.jpg'
# url = 'https://perevozka24.ru/img/ck_upload/kamaz-53212.jpg'
# url = 'https://motor.ru/thumb/1500x0/filters:quality(75):no_upscale()/imgs/2020/12/25/13/4425089/1d6587c59ae101528b6a68d2d11ecefe104e3ea2.jpg'
url = 'https://drikus.club/uploads/posts/2022-01/1643106751_1-drikus-club-p-kamaz-mchs-novii-tekhnika-krasivo-foto-1.jpg'
url = 'https://static.ngs.ru/news/99/preview/a3ca1d2daa15a599ea2a96cc26e86dd40904e3c2_1088_725_c.jpg'
url = 'https://file.lact.ru/f1/s/8/66/image/1554/50/medium_head_automatic-road-barriers_en.jpg?t=1482745951'
url = 'https://katlavan.ru/img/ustanovka-shlagbauma-vo-dvore-zhilogo-doma-zakon_0.jpg'


# качаем изображение
bin_img = requests.get(url, stream=True).raw

image = Image.open(bin_img)

# прогоняем через модель
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
# sequence_output = outputs.last_hidden_state
logits = outputs.logits

# преобразуем логитсы к размеру изображения
upsampled_logits = nn.functional.interpolate(
    logits,
    size=image.size[::-1], # (height, width)
    mode='bilinear',
    align_corners=False
)

# выбраем сработавший класс
pred_seg = upsampled_logits.argmax(dim=1)[0]

In [ ]:

plt.rcParams["figure.figsize"] = (50,80)
pred_img, color_seg = get_seg_overlay(image, pred_seg)
plt.imshow(pred_img)

print(color_seg)